# DoggieTech Dog Image Classifier
An application used to identify a dog breed.

**NOTE**: Currently, the algorithm only supports 3 unique breeds:
- Rottweilers
- Golden Retrievers
- Chihuahuas

You can also try out the examples provided in the _toClassify_ folder.

## How to use this application
If you are using this application **remotely** (i.e., Jupyter Binder, Google Colab), please follow the steps in [Section 1](#section-1-pre-use-setup), then view the [User Guide](./UserGuide.md) for the alternative uploading method and interaction. 

If running the application **locally**, please follow the steps in [Section 1](#section-1-pre-use-setup), then follow the steps in [Section 2](#section-2-using-the-application).

This is the interactive application where you submit an image of a dog for breed identification. Please skip to [Section 1](#section-1-pre-use-setup) for instructions to follow before using the application. 

<!-- If you already know how to use the application, please skip to [Section 2](#section-2-using-the-application). -->

## Section 0: Application Setup
No need to worry about this section. This is where the application setup occurs. After performing "Run All", jump to [Section 1](#section-1-pre-use-setup) to start interacting with the application.

In [ ]:
# Python imports
import ipywidgets as widgets
from ipywidgets import Layout, GridBox
import os
import pickle
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Loading the trained model
mlModel = pickle.load(open('./trained_models.p', 'rb'))
randForestModel = mlModel['rfc']
svcModel = mlModel['svc']

In [ ]:
# For formatting the images

IMG_SIZE = 100
MAX_CATEGORY_SIZE = 3

def stringToImg(imgPath: str):
  myImg = os.path.join(imgPath)
  temp = Image.open(myImg)
  myImg = temp
  myImg = resizeImage(temp, IMG_SIZE)
  
  temp.close()
  return myImg

def completeTransformation(imgPath: str):
  myImg = stringToImg(imgPath=imgPath)
  myImg = to24Bit(myImg)
  myImg = np.array(myImg).flatten()
  return myImg

def convertImgToNpArray(anImage: Image):
  return np.array(anImage)

def cropImage(anImage, boundingBox: tuple):
  return anImage.crop(boundingBox)

def resizeImage(anImage: Image, aSize: int):
  return anImage.resize((aSize, aSize))

def to24Bit(anImage: Image):
  return anImage.convert('L')

def displayImage(anImageArray: np.array, labelReal: str, labelPredicted: str, verbose = False):
  fig, ax = plt.subplots(1, 1)
  ax.set(xticks=[], yticks=[])
  ax.imshow(anImageArray)

  if(verbose):
    ax.set(xlabel=f'Predicted: {labelPredicted}\nActual: {labelReal}')
  else:  
    ax.set(xlabel=f'Predicted: {labelPredicted}')
  

In [ ]:
# To perform bulk classification
TO_ID_PATH = './toClassify/'
imageAcceptedTypes = ['jpg','jpeg','png']

toClassifyList = os.listdir(TO_ID_PATH)
inputSet = []
labelSet = []
imageSet = []
for image in toClassifyList:
  if(image.split('.')[-1] in imageAcceptedTypes):
    regImage = Image.open(os.path.join(TO_ID_PATH, image), 'r')
    label = ((image.split('.')[0]).split('0')[0]).strip('_').capitalize()
    labelSet.append(label)
    formattedImage = resizeImage(cropImage(anImage=regImage, boundingBox=regImage.getbbox()), IMG_SIZE)
    formattedImage24Bit = to24Bit(formattedImage)
    regImage.close()
    imageSet.append(formattedImage)
    inputSet.append(np.array(formattedImage24Bit).flatten())
  else:
    continue

In [ ]:
# Perform a prediction on the provided image
def analyzeImage(aFormattedImage):
  return randForestModel.predict([aFormattedImage])

## Section 1: Pre-use Setup
This is where you will walk through the the process of using this application. The steps are as follows:
- Take a picture of the dog.
- Crop the photo _(optional, but may help with accuracy)_.
- Upload the photo to the folder `toClassify`.
- Click the **Analyze Dog** button.

### Part A: Take a photo
Take a picture of the dog you'd like to analyze. The clearer the photo of the dog, the likelihood of the algorithm correctly identifying the dog breed increases. 

Here's an example of a good photo:
<figure>
<img src="./good_example.jpg" width="300" alt="Good photo of dog">
<figcaption>A clear photo of the dog showing distinct features.</figcaption>
</figure>
<br>
<!-- ![Good Photo of Dog](good_example.jpg "Something") -->


Here's an example of a bad photo:
<figure>
<img src="./bad_example.jpg" alt="Bad photo of dog" width="300">
<figcaption>The dog is obstructed by objects in the photo.</figcaption>
</figure>
<br>
<!-- ![Bad Photo of Dog](bad_example.jpg) -->

**Have a picture ready? Move onto [Part B](#part-b-crop-a-photo-optional)**

### Part B: Crop a photo (optional)
While cropping the photo is optional, it will help increase the accuracy of the photo analysis. ([Skip this step](#section-2-using-the-application))

Here's an example of an uncropped photo:
<figure>
<img src="./uncropped_example.jpeg" width="300" alt="An uncropped photo">
<figcaption>An uncropped photo of a dog and excessive background space.</figcaption>
</figure>
<br>



Here's an example of a cropped photo:
<figure>
<img src="./cropped_example.jpeg" alt="A cropped photo" width="300">
<figcaption>A cropped photo focused on the dog, reduced background space.</figcaption>
</figure>
<br>

**Have the cropped photo ready? ([Continue to Section 2](#section-2-using-the-application))**

## Section 2: Using the Application
### Upload a photo
Click on the button labeled **Upload** and select the photo you plan to use. Currently, this application only accepts '.jpg', '.jpeg', and '.png'. 

### Analyze the photo
Finally, we'll analyze the photo of the dog to identify its breed. Click on the **Analyze Dog** button. You'll see the image you've selected and the predicted breed the program has produced.

**NOTE**: Currently, the algorithm only supports 3 unique breeds:
- Rottweilers
- Golden Retrievers
- Chihuahuas

You can also try out the examples provided in the _toClassify_ folder.

In [ ]:
DEFAULT_PATH = './toClassify'

uploader = widgets.FileUpload(accept=".jpg,.jpeg,.png", multiple=False, layout=Layout(height='auto', grid_area='uploader'), description="Upload Photo")


button = widgets.Button(
  description="Analyze Dog",
  layout=Layout(height='auto', grid_area='button')
)

selection = widgets.Dropdown(
  description = f'Select file from "{DEFAULT_PATH}":',
  options= os.listdir(DEFAULT_PATH),
  layout=Layout(height='auto', grid_area='selection'),
  style={'description_width': 'initial'}

)

button.style.button_color = 'lightblue'

def on_button_click(b):
  try:
    userImg = uploader.value.keys()
    userImg = list(userImg)[0]
    anImg = os.path.join(userImg, "")
    print(anImg)
    uploader.style.button_color = 'lightgreen'
    displayImage(stringToImg(anImg), "", analyzeImage(completeTransformation(anImg)))
  except AttributeError as attrErr:
    uploader.style.button_color = 'red'
    print("Unable to locate image.")
  except:
    print(f'Using alternative photo in folder: {DEFAULT_PATH}')
    uploader.style.button_color = 'red'
    anImg = os.path.join(DEFAULT_PATH, selection.value)
    selection.style.button_color = 'lightgreen'
    displayImage(stringToImg(anImg), "", analyzeImage(completeTransformation(anImg)))



def on_file_upload(b):
  print(uploader.value)

gui = GridBox(children=[uploader, button, selection],
        layout=Layout(
          width='100%',
          grid_template_rows='50px 50px',
          padding='10px',
          grid_template_columns = '',
          grid_template_areas = '''
          "selection selection"
          "uploader button"
          '''
        ))
display(gui)
button.on_click(on_button_click)

## Section 3: Statistics
Interested in learning more about the model? Take a look at **Section 4** in the [README](./README.md) to see the statistics of the model, including the data set distribution, the classifiers used, etc.